In [27]:
import pickle

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm

from pt_runner.cnn import CheckpointHandler, DataHandlerPT, EarlyStopper

In [28]:
# New run
NEW_RUN = True
DT_REF = None

# Resuming
# NEW_RUN = False
# DT_REF = "2025-05-25_08-58"

In [29]:
RANDOM_STATE = 0

In [30]:
with open("mnist_small.pickle", "rb") as file:
    data = pickle.load(file)

In [31]:
_X = data["_X"].astype(float)
_Y = data["_Y"].astype(float)
print(_X.shape)
print(_X.dtype)
print(_Y.shape)
print(_Y.dtype)

(2000, 1, 28, 28)
float64
(2000, 1)
float64


In [32]:
data_handler = DataHandlerPT(_X=_X, _Y=_Y)

In [33]:
import torch.nn as nn
import torch.nn.functional as F


class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        # First convolutional layer: input channels=3 (e.g., RGB), output channels=16, kernel size=3
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        
        self.max_pool = nn.MaxPool2d(2)
        self.relu = nn.ReLU()
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        # Fully connected layer
        self.fc1 = nn.Linear(32 * 4 * 4, num_classes)  # Adjust input size depending on image size!

    def forward(self, X):
        # 1st Conv + Activation + Pooling
        X = self.conv1(X)
        X = self.relu(X)
        X = self.max_pool(X)
        # 2nd Conv + Activation + Pooling
        X = self.conv2(X)
        X = self.relu(X)
        X = self.max_pool(X)

        X = self.adaptive_pool(X)
        # Flatten the output for the fully connected layer
        X = X.view(X.shape[0], -1)
        # Fully connected output
        X = self.fc1(X)
        return X

model = SimpleCNN(num_classes=10)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=5)
loss_fn = nn.CrossEntropyLoss()


# Test
X = torch.randn(10,1, 28,28)
model(X)

tensor([[-4.9465e-02, -1.9132e-01, -1.4498e-01,  9.3011e-02,  2.1806e-01,
         -1.6085e-01, -4.5651e-02, -3.1434e-02, -1.7815e-01, -8.9619e-02],
        [-7.1117e-02, -1.6171e-01, -1.4174e-01,  1.4484e-01,  2.8498e-01,
         -1.5732e-01,  1.3213e-02, -2.0569e-04, -1.6518e-01, -6.5073e-02],
        [-3.4264e-02, -9.3897e-02, -1.1449e-01,  4.3201e-02,  1.9634e-01,
         -1.2571e-01, -2.3975e-02, -5.9899e-02, -1.7136e-01, -4.4362e-02],
        [-6.9071e-03, -1.4762e-01, -7.4263e-02,  1.0062e-01,  2.5548e-01,
         -1.3183e-01,  5.9588e-03, -2.7009e-02, -1.1527e-01, -1.1156e-01],
        [-8.1074e-03, -1.8677e-01, -3.9886e-02,  1.1900e-01,  2.7171e-01,
         -1.4307e-01, -2.6917e-04,  6.2861e-03, -1.4744e-01, -4.7850e-02],
        [-1.4945e-01, -7.0126e-02, -8.9559e-02,  5.0698e-02,  2.8521e-01,
         -1.3894e-01, -2.4447e-03, -5.5963e-02, -1.4848e-01, -1.2180e-01],
        [-7.8281e-02, -1.7695e-01, -1.0124e-01,  9.2421e-02,  2.3565e-01,
         -1.4593e-01, -1.4129e-0

In [34]:
from torchinfo import summary

input_size = (100, 1, 32, 32) # (batch_size, channels, height, width)
summary(model, input_size=input_size)

Layer (type:depth-idx)                   Output Shape              Param #
SimpleCNN                                [100, 10]                 --
├─Conv2d: 1-1                            [100, 16, 32, 32]         160
├─ReLU: 1-2                              [100, 16, 32, 32]         --
├─MaxPool2d: 1-3                         [100, 16, 16, 16]         --
├─Conv2d: 1-4                            [100, 32, 16, 16]         4,640
├─ReLU: 1-5                              [100, 32, 16, 16]         --
├─MaxPool2d: 1-6                         [100, 32, 8, 8]           --
├─AdaptiveAvgPool2d: 1-7                 [100, 32, 4, 4]           --
├─Linear: 1-8                            [100, 10]                 5,130
Total params: 9,930
Trainable params: 9,930
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 135.68
Input size (MB): 0.41
Forward/backward pass size (MB): 19.67
Params size (MB): 0.04
Estimated Total Size (MB): 20.12

In [35]:
n_epochs = 1000  # number of epochs to run
batch_size = 10  # size of each batch
validation_interval = 10  # Evaluate every 100 epochs
log_name = "M1"
random_state = 0  # Split data

# Save/load
cph = CheckpointHandler()
cph.make_dir("./checkpoints")
if NEW_RUN:
    dt = cph.get_dt()
    log_dir = f"runs/{dt}"
    save_path = f"./checkpoints/{dt}.pth"
    epoch_start = 0
else:
    log_dir = f"runs/{DT_REF}"
    load_path = f"./checkpoints/{DT_REF}.pth"
    save_path = load_path
    model, optimizer, epoch, val_loss = cph.load(
        load_path=load_path, model=model, optimizer=optimizer
    )
    epoch_start = epoch
    print(f"Resuming from epoch: {epoch}")

epoch_end = epoch_start + n_epochs

# Initialize Components
early_stopper = EarlyStopper(patience=10)
writer = SummaryWriter(log_dir=log_dir)

# Data
data_handler.split_and_scale(test_size=0.2, val_size=0.1, random_state=RANDOM_STATE)
ds_train = data_handler.get_train()
ds_test = data_handler.get_test()
ds_val = data_handler.get_val()
loader_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(ds_val, batch_size=batch_size, shuffle=False)

# Main loop
for epoch in tqdm(
    range(epoch_start, epoch_end), initial=epoch_start, desc="Epoch", total=n_epochs
):
    # Training Phase
    model.train()
    epoch_train_loss = 0.0

    for X_batch, Y_batch in loader_train:
        optimizer.zero_grad()
        Y_pred = model(X_batch)
        loss = loss_fn(Y_pred, Y_batch)
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        # Update weights
        optimizer.step()
        # Multiplies the average loss per sample by the number of
        # samples in the batch to get the total loss for this batch.
        epoch_train_loss += loss.item() * X_batch.size(0)

    avg_train_loss = epoch_train_loss / len(loader_train.dataset)

    # Validation Phase
    if epoch % validation_interval == 0 or epoch == epoch_start:
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_val, Y_val in loader_val:
                Y_pred = model(X_val)
                val_loss += loss_fn(Y_pred, Y_val).item() * X_val.size(0)

        avg_val_loss = val_loss / len(loader_val.dataset)
        scheduler.step(avg_val_loss)

        # Early Stopping and Checkpoint
        es = early_stopper(avg_val_loss)
        if es["best_loss"]:
            cph.save(
                save_path=save_path,
                model=model,
                optimizer=optimizer,
                val_loss=avg_val_loss,
                epoch=epoch,
            )
            print("Save model @ epoch:", epoch)
        if es["early_stop"]:
            print("Stopped at epoch:", epoch)
            break

        writer.add_scalars(
            log_name, {"train_loss": avg_train_loss, "val_loss": avg_val_loss}, epoch
        )


Epoch:   0%|          | 0/1000 [00:00<?, ?it/s]

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [39]:
Y_batch.shape

torch.Size([10, 1])

In [ ]:
X_batch.shape

torch.Size([10, 1, 28, 28])